#Install libraries

In [1]:
!pip install transformers
!pip install torch torchvision
!pip install pandas
!pip install numpy
!pip install datasets
!pip install pytorch_transformers
!pip install scikit-learn
!pip install matplotlib
!pip install seaborn
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 3.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

Import the required libraries

In [2]:
import torch
from torch.utils.data import (TensorDataset, DataLoader,
                              RandomSampler, SequentialSampler)

from pytorch_transformers import BertTokenizer, BertConfig
from pytorch_transformers import BertForSequenceClassification
from pytorch_transformers import AdamW, WarmupLinearSchedule

from distutils.version import LooseVersion as LV

from sklearn.model_selection import train_test_split

import io

import pandas as pd
import numpy as np

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns

from datasets import load_dataset

import tensorflow_datasets as tfds

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


sns.set()

if torch.cuda.is_available():
    device = torch.device('cuda')
    devicename = '['+torch.cuda.get_device_name(0)+']'
else:
    device = torch.device('cpu')
    devicename = ""
    
print('Using PyTorch version:', torch.__version__,
      'Device:', device, devicename)
assert(LV(torch.__version__) >= LV("1.0.0"))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using PyTorch version: 1.13.1+cu116 Device: cuda [Tesla T4]


Download the IMDb and SST-2 datasets and extract them.

In [3]:
# Load the IMDB dataset
imdb_dataset = load_dataset("imdb")


# Load the SST-2 dataset
sst2_dataset = load_dataset("glue", "sst2")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Load the IMDb dataset using pandas, and preprocess the text data by removing HTML tags, non-alphanumeric characters, and stop words.

In [4]:
print(imdb_dataset.column_names)

{'train': ['text', 'label'], 'test': ['text', 'label'], 'unsupervised': ['text', 'label']}


In [5]:
# Load the IMDb dataset
imdb_train_df = pd.DataFrame(imdb_dataset['train'])
imdb_train_df = imdb_train_df.reset_index(drop=True)
imdb_test_df = pd.DataFrame(imdb_dataset['test'])
imdb_test_df = imdb_test_df.reset_index(drop=True)

print('\nIMDB data loaded:')
print('train:', imdb_train_df.shape)
print('test:', imdb_test_df.shape)
print(imdb_train_df['label'].unique())
print(imdb_test_df['label'].unique())


IMDB data loaded:
train: (25000, 2)
test: (25000, 2)
[0 1]
[0 1]


In [6]:
# Let's view some random reviews:
print(imdb_train_df.sample(5))
print(imdb_test_df.sample(5))

                                                    text  label
8924   This film, by Oscar Petersson, is unique. Its ...      0
10730  I'd never seen a Tarzan movie before so when I...      0
16792  A milestone in Eastern European film making an...      1
6433   I love Anthony Hopkins as an actor so I was ve...      0
23379  This was the most visually stunning, moving, a...      1
                                                    text  label
4844   I really, really don't understand how that mov...      0
17153  Used to watch this when i was very little, the...      1
1068   One of the few reasons to make these pointlss ...      0
20467  2002's the Bourne Identity is one of my all-ti...      1
2106   Joe was first released in the US in the summer...      0


In [7]:
# Preprocess the text data
imdb_train_df ['text'] = imdb_train_df ['text'].str.replace('<.*?>', '', regex=True) # remove HTML tags
imdb_train_df ['text'] = imdb_train_df ['text'].str.replace('[^a-zA-Z0-9\s]', '', regex=True) # remove non-alphanumeric characters
stop_words = set(stopwords.words('english'))
imdb_train_df ['text'] = imdb_train_df ['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words])) # remove stop words


imdb_test_df ['text'] = imdb_test_df ['text'].str.replace('<.*?>', '', regex=True) # remove HTML tags
imdb_train_df ['text'] = imdb_test_df ['text'].str.replace('[^a-zA-Z0-9\s]', '', regex=True) # remove non-alphanumeric characters
stop_words = set(stopwords.words('english'))
imdb_test_df ['text'] = imdb_test_df ['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words])) # remove stop words

Load the SST-2 dataset using pandas, and preprocess the text data in the same way as the IMDb dataset

In [8]:
print(sst2_dataset.column_names)

{'train': ['sentence', 'label', 'idx'], 'validation': ['sentence', 'label', 'idx'], 'test': ['sentence', 'label', 'idx']}


In [9]:
# Load the SST-2 dataset
sst2_train_df = pd.DataFrame(sst2_dataset['train'])[['sentence', 'label']]
sst2_train_df = sst2_train_df.rename(columns={'sentence': 'text'})
sst2_train_df = sst2_train_df.reset_index(drop=True)

sst2_test_df = pd.DataFrame(sst2_dataset['validation'])[['sentence', 'label']]
sst2_test_df = sst2_test_df.rename(columns={'sentence': 'text'})
sst2_test_df = sst2_test_df.reset_index(drop=True)

#sst2_test_df =pd.concat([pd.DataFrame(sst2_dataset['test'])[['sentence', 'label']], pd.DataFrame(sst2_dataset['validation'])[['sentence', 'label']]])

print('\nSST2 data loaded:')
print('train:', sst2_train_df.shape)
print('test:', sst2_test_df.shape)
print(sst2_train_df['label'].unique())
print(sst2_test_df['label'].unique())


SST2 data loaded:
train: (67349, 2)
test: (872, 2)
[0 1]
[1 0]


Unbalanced set. Replace default split by train_test_split

In [10]:
# Load your dataframe
df = pd.concat([sst2_train_df,sst2_test_df])

# Define your features and target variable
X = df.drop("label", axis=1)
y = df["label"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shape of the train and test sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

sst2_train_df = pd.concat([X_train,y_train], axis=1)
sst2_test_df = pd.concat([X_test,y_test], axis=1)

print('\nSST2 data re splitted:')
print('train:', sst2_train_df.shape)
print('test:', sst2_test_df.shape)
print(sst2_train_df['label'].unique())
print(sst2_test_df['label'].unique())


X_train shape: (54576, 1)
X_test shape: (13645, 1)
y_train shape: (54576,)
y_test shape: (13645,)

SST2 data re splitted:
train: (54576, 2)
test: (13645, 2)
[1 0]
[1 0]


In [11]:
# Let's view some random reviews:
print(sst2_train_df.sample(5))
print(sst2_test_df.sample(5))

                                                    text  label
65349  such master screenwriting comes courtesy of jo...      1
61054                                 embarrassment and       0
40326                            wonderfully loopy tale       1
27672  a fine effort , an interesting topic , some in...      1
10554  it 's light on the chills and heavy on the atm...      1
                                                    text  label
28323  comes from the brave , uninhibited performance...      1
5146   will make you think twice about what might be ...      1
57173  interview '' loses its overall sense of myster...      0
31117                         a funny and touching film       1
9620   a good race , one that will have you at the ed...      1


In [12]:
# Preprocess the text data
sst2_train_df['text'] = sst2_train_df['text'].str.replace('<.*?>', '', regex=True) # remove HTML tags
sst2_train_df['text'] = sst2_train_df['text'].str.replace('[^a-zA-Z0-9\s]', '', regex=True) # remove non-alphanumeric characters
stop_words = set(stopwords.words('english'))
sst2_train_df['text'] = sst2_train_df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words])) # remove stop words


sst2_test_df['text'] = sst2_test_df['text'].str.replace('<.*?>', '', regex=True) # remove HTML tags
sst2_test_df['text'] = sst2_test_df['text'].str.replace('[^a-zA-Z0-9\s]', '', regex=True) # remove non-alphanumeric characters
stop_words = set(stopwords.words('english'))
sst2_test_df['text'] = sst2_test_df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words])) # remove stop words


Training and testing on SST2, OOD detection on IMDB

In [16]:
train_df = sst2_train_df
test_df = sst2_test_df
imdb_df = pd.concat([imdb_train_df,imdb_test_df])
n_ood = 0.1
ood_df = imdb_df.sample(int(n_ood*train_df.shape[0]))
ood_df = ood_df.reset_index(drop=True)

In [18]:
# Let's view some random reviews:
print(train_df.sample(5))
print(test_df.sample(5))
print(ood_df.sample(5))

                                                    text  label
44795                                         indulgence      0
7735                                              limpid      0
34839                                   worst films 2002      0
23399  guess space station year 2455 crossed list ide...      0
52053     even stuffiest cinema goers laugh hourandahalf      1
                                                    text  label
31064  shrill soporific everything repeated five six ...      0
53548           birthday girl actor movie first foremost      1
47555             feel seeing something purer real thing      1
45008                                simply intoxicating      1
27891                                                fit      1
                                                   text  label
4811  Alexander Nevsky marked director Sergei Eisens...      1
5441  OK...before I even start less constructive cri...      0
1049  I rented this movie today worst movie

The token `[CLS]` is a special token required by BERT at the beginning of the sentence.

In [19]:
sentences_train = train_df.text.values
sentences_train = ["[CLS] " + s for s in sentences_train]

sentences_test = test_df.text.values
sentences_test = ["[CLS] " + s for s in sentences_test]

sentences_ood = ood_df.text.values
sentences_ood = ["[CLS] " + s for s in sentences_ood]


labels_train = train_df.label.values
labels_test  = test_df.label.values
labels_ood  = ood_df.label.values

print ("\nThe first training sentence:")
print(sentences_train[0], 'LABEL:', labels_train[0])



The first training sentence:
[CLS] wildly alive LABEL: 1


Next we use the BERT tokenizer to convert the sentences into tokens
that match the data BERT was trained on.


In [24]:
BERTMODEL = "bert-base-uncased"

tokenizer = BertTokenizer.from_pretrained(BERTMODEL,
                                          do_lower_case=True)

tokenized_train = [tokenizer.tokenize(s) for s in sentences_train]
tokenized_test  = [tokenizer.tokenize(s) for s in sentences_test]
tokenized_ood  = [tokenizer.tokenize(s) for s in sentences_ood]

print ("\nThe full tokenized first training sentence:")
print (tokenized_train[0])

print ("\nThe full tokenized first test sentence:")
print (tokenized_test[0])

print ("\nThe full tokenized first OOD sentence:")
print (tokenized_ood[0])


The full tokenized first training sentence:
['[CLS]', 'wildly', 'alive']

The full tokenized first test sentence:
['[CLS]', 'best', 'script']

The full tokenized first OOD sentence:
['[CLS]', 'dreadful', 'a', 'friend', 'of', 'mine', 'who', 'obviously', 'thought', 'i', 'had', 'an', 'ab', '##ys', '##mal', 'sense', 'of', 'humour', 'recommended', 'this', '##its', 'bob', '##bin', '##s', 'i', 'almost', 'switched', 'it', 'off', 'it', 'is', 'only', 'my', 'anal', 'desire', 'to', 'not', 'leave', 'things', 'unfinished', 'that', 'prevented', 'me', 'doing', 'so', '##thi', '##s', 'was', 'evidently', 'a', 'british', 'attempt', 'to', 'make', 'a', 'movie', 'with', 'a', 'bunch', 'of', 'also', 'ran', 'tv', 'actors', 'using', 'some', 'lame', 'script', 'from', 'their', 'mate', 'in', 'the', 'business', 'i', 'struggle', 'to', 'think', 'of', 'anything', 'even', 'approaching', 'the', 'pau', '##city', 'of', 'this', 'movie', 'less', 'funny', 'than', 'global', 'warming', '##im', 'not', 'normally', 'so', 've', '#


Now we set the maximum sequence lengths for our training and test
sentences as `MAX_LEN_TRAIN` and `MAX_LEN_TEST`. The maximum length
supported by the used BERT model is 512.

The token `[SEP]` is another special token required by BERT at the
end of the sentence.

In [25]:
MAX_LEN_TRAIN, MAX_LEN_TEST = 128, 512

tokenized_train = [t[:(MAX_LEN_TRAIN-1)]+['SEP'] for t in tokenized_train]
tokenized_test  = [t[:(MAX_LEN_TEST-1)]+['SEP'] for t in tokenized_test]
tokenized_ood  = [t[:(MAX_LEN_TEST-1)]+['SEP'] for t in tokenized_ood]

print ("\nThe truncated tokenized first training sentence:")
print (tokenized_train[0])


The truncated tokenized first training sentence:
['[CLS]', 'wildly', 'alive', 'SEP']



Next we use the BERT tokenizer to convert each token into an integer
index in the BERT vocabulary. We also pad any shorter sequences to
`MAX_LEN_TRAIN` or `MAX_LEN_TEST` indices with trailing zeros.

In [26]:
ids_train = [tokenizer.convert_tokens_to_ids(t) for t in tokenized_train]
ids_train = np.array([np.pad(i, (0, MAX_LEN_TRAIN-len(i)),
                             mode='constant') for i in ids_train])

ids_test = [tokenizer.convert_tokens_to_ids(t) for t in tokenized_test]
ids_test = np.array([np.pad(i, (0, MAX_LEN_TEST-len(i)),
                            mode='constant') for i in ids_test])


ids_ood = [tokenizer.convert_tokens_to_ids(t) for t in tokenized_ood]
ids_ood = np.array([np.pad(i, (0, MAX_LEN_TEST-len(i)),
                            mode='constant') for i in ids_ood])

print ("\nThe indices of the first training sentence:")
print (ids_train[0])


The indices of the first training sentence:
[  101 13544  4142   100     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]


BERT also requires *attention masks*, with 1 for each real token in
the sequences and 0 for the padding:

In [27]:
amasks_train, amasks_test , amasks_ood = [], [] , []

for seq in ids_train:
  seq_mask = [float(i>0) for i in seq]
  amasks_train.append(seq_mask)

for seq in ids_test:
  seq_mask = [float(i>0) for i in seq]
  amasks_test.append(seq_mask)


for seq in ids_ood:
  seq_mask = [float(i>0) for i in seq]
  amasks_ood.append(seq_mask)

We use scikit-learn's train_test_split() to use 10% of our training
data as a validation set, and then convert all data into
torch.tensors.

In [28]:
(train_inputs, validation_inputs,
 train_labels, validation_labels) = train_test_split(ids_train, labels_train,
                                                     random_state=42,
                                                     test_size=0.1)
(train_masks, validation_masks,
 _, _) = train_test_split(amasks_train, ids_train,
                          random_state=42, test_size=0.1)

train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks  = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks  = torch.tensor(validation_masks)
test_inputs = torch.tensor(ids_test)
test_labels = torch.tensor(labels_test)
test_masks  = torch.tensor(amasks_test)
ood_inputs = torch.tensor(ids_ood)
ood_labels = torch.tensor(labels_ood)
ood_masks  = torch.tensor(amasks_ood)



Next we create PyTorch *DataLoader*s for all data sets.
For fine-tuning BERT on a specific task, the authors recommend a
batch size of 16 or 32.

In [29]:
BATCH_SIZE = 32

print('\nDatasets:')
print('Train: ', end="")
train_data = TensorDataset(train_inputs, train_masks,
                           train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler,
                              batch_size=BATCH_SIZE)
print(len(train_data), 'reviews')

print('Validation: ', end="")
validation_data = TensorDataset(validation_inputs, validation_masks,
                                validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data,
                                   sampler=validation_sampler,
                                   batch_size=BATCH_SIZE)
print(len(validation_data), 'reviews')

print('Test: ', end="")
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler,
                             batch_size=BATCH_SIZE)
print(len(test_data), 'reviews')


print('OOD: ', end="")
ood_data = TensorDataset(ood_inputs, ood_masks, ood_labels)
ood_sampler = SequentialSampler(ood_data)
ood_dataloader = DataLoader(ood_data, sampler=ood_sampler,
                             batch_size=BATCH_SIZE)
print(len(ood_data), 'reviews')


Datasets:
Train: 49118 reviews
Validation: 5458 reviews
Test: 13645 reviews
OOD: 5457 reviews


BERT MODEL INITIALIZATION

We now load a pretrained BERT model with a single linear
classification layer added on top.


In [30]:
model = BertForSequenceClassification.from_pretrained(BERTMODEL,
                                                      num_labels=2)

model.cuda()
print('\nPretrained BERT model "{}" loaded'.format(BERTMODEL))

100%|██████████| 440473133/440473133 [00:37<00:00, 11809454.23B/s]



Pretrained BERT model "bert-base-uncased" loaded



We set the remaining hyperparameters needed for fine-tuning the
pretrained model: 
 * EPOCHS: the number of training epochs in fine-tuning
   (recommended values between 2 and 4) 
 * WEIGHT_DECAY: weight decay for the Adam optimizer 
 * LR: learning rate for the Adam optimizer 
   (2e-5 to 5e-5 recommended) 
 * WARMUP_STEPS: number of warmup steps to (linearly) reach the
   set learning rate

 We also need to grab the training parameters from the pretrained
 model.

In [31]:
EPOCHS = 4
WEIGHT_DECAY = 0.01
LR = 2e-5
WARMUP_STEPS =int(0.2*len(train_dataloader))

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters()
                if not any(nd in n for nd in no_decay)],
     'weight_decay': WEIGHT_DECAY},
    {'params': [p for n, p in model.named_parameters()
                if any(nd in n for nd in no_decay)],
     'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=LR, eps=1e-8)
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=WARMUP_STEPS,
                                 t_total=len(train_dataloader)*EPOCHS)

LEARNING

Let's now define functions to train() and evaluate() the model:

In [32]:
def train(epoch, loss_vector=None, log_interval=200):
  # Set model to training mode
  model.train()

  # Loop over each batch from the training set
  for step, batch in enumerate(train_dataloader):

    # Copy data to GPU if needed
    batch = tuple(t.to(device) for t in batch)

    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch

    # Zero gradient buffers
    optimizer.zero_grad()

    # Forward pass
    outputs = model(b_input_ids, token_type_ids=None,
                    attention_mask=b_input_mask, labels=b_labels)

    loss = outputs[0]
    if loss_vector is not None:
        loss_vector.append(loss.item())

    # Backward pass
    loss.backward()

    # Update weights
    scheduler.step()
    optimizer.step()

    if step % log_interval == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, step * len(b_input_ids),
                len(train_dataloader.dataset),
                100. * step / len(train_dataloader), loss))

def evaluate(loader):
  model.eval()

  n_correct, n_all = 0, 0

  for batch in loader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids=None,
                      attention_mask=b_input_mask)
      logits = outputs[0]

    logits = logits.detach().cpu().numpy()
    predictions = np.argmax(logits, axis=1)

    labels = b_labels.to('cpu').numpy()
    n_correct += np.sum(predictions == labels)
    n_all += len(labels)

  print('Accuracy: [{}/{}] {:.4f}'.format(n_correct, n_all,
                                          n_correct/n_all))

Now we are ready to train our model using the train()
function. After each epoch, we evaluate the model using the
validation set and evaluate().

In [33]:
train_lossv = []
for epoch in range(1, EPOCHS + 1):
    print()
    train(epoch, train_lossv)
    print('\nValidation set:')
    evaluate(validation_dataloader)

/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/usr/local/lib/python3.8/dist-packages/pytorch_transformers/optimization.py:166: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1420.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


Train Epoch: 1 [0/49118 (0%)]	Loss: 0.709738
Train Epoch: 1 [6400/49118 (13%)]	Loss: 0.389463
Train Epoch: 1 [12800/49118 (26%)]	Loss: 0.313381
Train Epoch: 1 [19200/49118 (39%)]	Loss: 0.277281
Train Epoch: 1 [25600/49118 (52%)]	Loss: 0.441624
Train Epoch: 1 [32000/49118 (65%)]	Loss: 0.432275
Train Epoch: 1 [38400/49118 (78%)]	Loss: 0.163827
Train Epoch: 1 [44800/49118 (91%)]	Loss: 0.319243

Validation set:
Accuracy: [4979/5458] 0.9122

Train Epoch: 2 [0/49118 (0%)]	Loss: 0.100935
Train Epoch: 2 [6400/49118 (13%)]	Loss: 0.161336
Train Epoch: 2 [12800/49118 (26%)]	Loss: 0.098466
Train Epoch: 2 [19200/49118 (39%)]	Loss: 0.122900
Train Epoch: 2 [25600/49118 (52%)]	Loss: 0.148851
Train Epoch: 2 [32000/49118 (65%)]	Loss: 0.161891
Train Epoch: 2 [38400/49118 (78%)]	Loss: 0.190154
Train Epoch: 2 [44800/49118 (91%)]	Loss: 0.259956

Validation set:
Accuracy: [5066/5458] 0.9282

Train Epoch: 3 [0/49118 (0%)]	Loss: 0.085955
Train Epoch: 3 [6400/49118 (13%)]	Loss: 0.009738
Train Epoch: 3 [12800/49

Let's take a look at our training loss over all batches:

In [34]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15,8))
plt.title("Training loss")
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.plot(train_lossv, label='original')
plt.plot(np.convolve(train_lossv, np.ones(101), 'same') / 101,
         label='averaged')
plt.legend(loc='best')
plt.savefig("training-loss.png")
plt.show()


Inference

For a better measure of the quality of the model, let's see the
model accuracy for the test reviews.

In [35]:
print('\nTest set:')
evaluate(test_dataloader)

# eof


Test set:
Accuracy: [12720/13645] 0.9322


In [36]:
print('\nOod set:')
evaluate(ood_dataloader)

# eof


Ood set:
Accuracy: [4701/5457] 0.8615


Once training is complete, we can evaluate the model on the SST2 dataset:

Finally, we'll save the hidden layers and trained values of the model:
#TO DO

In [ ]:
# Save hidden layers and trained values
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'hidden_layers': model.encoder.layer[-1].output_hidden_states,
    'trained_values': model.pooler.dense.weight
}, 'path/to/save/model.pt')
